In [ ]:
from detection import IcebergDetector, IcebergDetectionConfig
from embedding import IcebergEmbeddingsTrainer, IcebergEmbeddingsConfig
from tracking import IcebergTracker, IcebergTrackingConfig
from utils.visualize import Visualizer, VisualizationConfig
from utils.helpers import load_config

In [ ]:
train_dataset = "hill/train"
test_dataset = "hill/test"

In [ ]:
config = IcebergEmbeddingsConfig(dataset=train_dataset)         # Alternatively use YAML file: config = load_config(cfg_file="cfgs/embed.yaml", dataset=train_dataset)
trainer = IcebergEmbeddingsTrainer(config)
trainer.run_complete_pipeline()

In [ ]:
config = IcebergDetectionConfig(dataset=train_dataset)          # Alternatively use YAML file: config = load_config(cfg_file="cfgs/detect.yaml", dataset=train_dataset)
detector = IcebergDetector(config)
detector.train()

In [ ]:
config = IcebergDetectionConfig(dataset=test_dataset)           # Alternatively use YAML file: config = load_config(cfg_file="cfgs/detect.yaml", dataset=test_dataset)
detector = IcebergDetector(config)
detector.predict()

In [ ]:
config = IcebergTrackingConfig(dataset=test_dataset)            # Alternatively use YAML file: config = load_config(cfg_file="cfgs/track.yaml", dataset=test_dataset)
tracker = IcebergTracker(config)
tracker.track()

In [ ]:
config = VisualizationConfig(dataset=test_dataset)              # Alternatively use YAML file: config = load_config(cfg_file="cfgs/visualize.yaml", dataset=test_dataset)
visualizer = Visualizer(config)
visualizer.annotate_icebergs()
#visualizer.render_video()              # Uncomment to also render video